# Advanced Machine Learning - Practicum 6 - Convolutional Neural Network

**Topics covered**: Convolutional Neural Network

**Deliverables**:
- Complete the tasks as detailed in this document.  <br>
- The keras model file `mymodel.h5` (see Task 2 for detail). <br>
This practice uses [Keras](https://keras.io/api/) with Tensorflow.

**Objectives**:
After completing the tasks, you'll know how to:
- implement a simplified 7-layer LeNet CNN, and <br>
- apply it for object recognition. <br>
- design and implement a convnet model on a given data set. <br>

---
Import needed packages

In [ ]:
import numpy as np
import tensorflow as tf    # one-hot encode
from PIL import Image
from keras import layers
from keras import models
from keras.callbacks import EarlyStopping
from keras import optimizers
import matplotlib.pyplot as plt

## 1. Task 1 - Implement a convnet

### 1.1 Load image and Get train, test data set
The facial images are frm **olivettifaces** we used before. <br>
There are ten different images of each of 40 distinct subjects. The image is quantized to 256 grey levels and stored as unsigned 8-bit integers. Size of each image is 57 by 47 with 1 channel.<br>
In order to adapt to convnets, the data and target need to be reshaped. <br>
Run code in cell below to get `train_data`, `train_target`, `test_data`, `test_target`

In [ ]:
# Load and normalize face image data
def load_images(filename):
    
    # open files, use Image.open()
    img = Image.open(filename)
    
    # Normalize pixle values, let it with range [0,1]
    img_ndarray = np.asarray(img, dtype='float32')/255
    
    # Split the entire image into 400 samples
    faces = np.empty((400,2679),dtype='float32')
    for row in range(20):
        for column in range(20):
            faces[row*20+column] = np.ndarray.flatten(img_ndarray[row*57:(row+1)*57,column*47:(column+1)*47])
            
    # Create target array    
    target = np.empty(400)
    for i in range(40):
        target[i*10:i*10+10]=i

    target=target.astype(np.int)
    
    #train, validation
    train_data=np.empty((360,2679),dtype='float32')
    train_target=np.empty(360)
    test_data=np.empty((40,2679),dtype='float32')
    test_target=np.empty(40)

    for i in range(40):
        train_data[i*9:i*9+9]=faces[i*10:i*10+9]
        train_target[i*9:i*9+9]=target[i*10:i*10+9]
        test_data[i]=faces[i*10+9]
        test_target[i]=target[i*10+9]
        
    # reshape
    train_data = train_data.reshape((360,57,47,1))
    test_data = test_data.reshape((40,57,47,1))
    
    # one-hot encode for target
    train_target = tf.keras.utils.to_categorical(train_target)
    test_target = tf.keras.utils.to_categorical(test_target)    
        
    return train_data, train_target, test_data, test_target

train_data, train_target, test_data, test_target = load_images("olivettifaces.gif")  


### 1.2 Build Convnet model
Implement function `build_CNN()` to build the convnet described as following: <br>
1)	Layer 0 - conv2d_1 (Conv2D), 5 kernels, size of kernel: 5 by 5, input of this layer is a 3D tesnor (57, 47, 1), activation function is tanh <br>
2)	Layer 1 - maxpooling2d_1 (MaxPooling2D), 2 by 2 pooling <br>
3)	Layer 2 - conv2d_2 (Conv2D), 10 kernels, size of kernel: 5 by 5, activation function is tanh <br>
4)	Layer 3 - maxpooling2d_2 (MaxPooling2D), 2 by 2 pooling <br>
5)	Layer 4 - flatten_1 (Flatten) <br>
6)	Layer 5 - dense_1 (Dense), 1000 nodes, activation function is tanh <br>
7)	Layer 6 - dense_2(Dense), 40 nodes, activation function is softmax


In [ ]:
def build_CNN():
    
    # Your code goes here

    return model

### 1.3 Train the convnet
- Call `build_CNN` get the convnet model
- Compile the modle (`compile()`, use the following hyperparameter setting to compile the model.<br>
    1) `optimizer=rmsprop` <br>
    2) `loss=categorical_crossentropy` <br>
    3) `metrics=[‘accuracy’]`. <br>
- Train the model (`fit()`). <br>
    1) Apply **early stopping** in the learning/fitting process. Click [here](https://chrisalbon.com/deep_learning/keras/neural_network_early_stopping/) for a introduction of very basic use of early stopping. <br>
    2) Plot the learning curve by using the returned result from `fit()`

In [ ]:
# Your code goes here


### 1.4 Plot learning curve on train loss

In [ ]:
# display the loss curves

# Your code goes here

### 1.5 Make prediction
Run the code below to evaluate the model and observe the prediction result.

In [ ]:
model.evaluate(test_data, test_target)
predictions = model.predict(test_data)

`predict` function returns a probability distribution over all 40 subjects. How do we get the identity from the `predctions`, type the statement in the cell below

In [ ]:
# predict the label
# Your code goes here


**How many subjects are wrongly classified? Give the identity of those subjects?**

Ans: **Type your answer here**

---

## 2. Task 2 - Build up your own convnet

In this task you're to build your own model and train it on the cats and dogs images. Save and submit your best modle (`model.save()` can be used to save your modle onto the disk).

### 2.1 Load Data
This task focus on classifying images as dogs or cats, in a dataset containing 4,000 pictures of cats and dogs (2,000 cats, 2,000 dogs). We’ll use 2,000 pictures for training—1,000 for validation, and 1,000 for testing. The original dataset could be downloaded from www.kaggle.com/c/dogs-vs-cats/data <br>
We'll use the split dataset which can be downloaded from the moodle course page.

In [ ]:
# specify the folder of train, validation and test data set
import os

# Change the follwing path if the dataset is not located here
train_dir = "../dogvscat/train"
validation_dir = "../dogvscat/validation"
test_dir = "../dogvscat/test"

train_cats_dir = os.path.join(train_dir, 'cats')  
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

In [ ]:
# Using ImageDataGenerator to read images from directories
# The generator yields batches of 150 × 150 RGB images (shape (20, 150, 150, 3)) and binary labels (shape (20,)).
# There are 20 samples in each batch (the batch size).

from keras.preprocessing.image import ImageDataGenerator

# Rescales all images by 1/255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='binary')

### 2.2 Bulid your convnet and Train it

Design and implement your own model for classifying cats and dogs images. Save the trained model into the file `mymodel.h5`. <br>
When calling `fit()`, `train_data, train_target` should be replace with `train_generator`, and use `validation_generator` to initialize parameter `validation_data`.

In [ ]:
# Your code goes here


In [ ]:
# run the code below to load and evaluate your model

model_stu = models.load_model('mymodel.h5')

# test_generator
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=20,
                                                  class_mode='binary')
model.evaluate(test_generator)
